In [237]:
import pandas as pd
from dateutil import parser
from datetime import date, timedelta
import os
import calendar

In [238]:
#!pip install openpyxl

## Target is to make all data weekly
### Period: Jan 2006 till Dec 2020

In [239]:
start_year=2006
end_year=2020

In [240]:
def find_financial_year_dates(start_financial_year, end_financial_year):
    start_date = date(start_financial_year,4,1)
    end_date = date(end_financial_year, 3, 31)
    
    delta = end_date - start_date
    dates = []
    for n in range(int (delta.days)+1):
        day = start_date + timedelta(n)
        dates.append(str(day)[0:10])
    return dates

In [241]:
def find_all_financial_year_dates(year_range):
    year0 = year_range.split('-')[0]
    year1 = year0[0:2]+year_range.split('-')[1]
    return find_financial_year_dates(int(year0), int(year1))

In [242]:
def find_year_dates(year):
    start_date = date(year,1,1)
    end_date = date(year, 12, 31)
    delta = end_date - start_date
    dates = []
    for n in range(int (delta.days)+1):
        day = start_date + timedelta(n)
        dates.append(str(day)[0:10])
    return dates

In [243]:
def find_dates(year, month):
    weekday, noofdays = calendar.monthrange(year, month)
    start_date = date(year,month,1)
    end_date = date(year, month, noofdays)
    delta = end_date - start_date
    dates = []
    for n in range(int (delta.days)+1):
        day = start_date + timedelta(n)
        dates.append(str(day)[0:10])
    return dates

In [244]:
def all_dates(start_year, end_year):
    dates = []
    for year in range(start_year, end_year+1):
        dates = dates+ find_year_dates(year)
    return dates

In [245]:
# Python's program to print all Monday's of a specific year
def find_all_mon_dates(start_year=start_year, end_year=end_year):
    all_years = range(start_year, end_year+1)
    all_mon_dates=[]
    for year in all_years:
        #print(year)
        date_object = date(year, 1, 1)
        if date_object.isoweekday() ==1:
            date_object += timedelta(days=1-date_object.isoweekday())
        else:
            date_object += timedelta(days=1-date_object.isoweekday())
            date_object += timedelta(days=7)
        #print(date_object)
        while date_object.year == year:
            #print(date_object)
            all_mon_dates.append(str(date_object)[0:10])
            date_object += timedelta(days=7)

    mon_date_df = pd.DataFrame(columns=['monday_date'])
    mon_date_df['monday_date'] = all_mon_dates
    mon_date_df.to_csv('monday_dates.csv')
    mon_date_df.info()
    return mon_date_df

In [246]:
def combine_data(directory):
    final_df = pd.DataFrame()
    for filename in os.listdir(directory):
        
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            #print(f)
            file_df = pd.read_excel(f)
            final_df = final_df.append(file_df)
    return final_df

In [247]:
def fill_gapes(df, date_column='date', start_year=start_year, end_year=end_year):
    print('Date Column: ', date_column)
    print('Start Year: ', start_year)
    print('End Year: ', end_year)
    dates = all_dates(start_year, end_year)
    prev_row=[]
    for date in dates:
        #print('Date: ', date)
        #print('Dataframe Date: ', df[date_column])
        row_data = df[df[date_column]==date]
        #print('Row Data#########',row_data)
        if(len(row_data)==0 and len(prev_row)>0):
            #print('Addingggggggggggg ', prev_row[date_column], 'as of ', date)
            prev_row[date_column] = date
            
            df = df.append(prev_row)
           
        if(len(row_data)>0):
            prev_row = row_data
            #print('Prev Row: ', prev_row)
    df.drop_duplicates(inplace=True)
    return df

In [248]:
### Collect Weekly Data
def filter_weekly_data(data_df, type, date_column):
    mon_dates = pd.read_csv('monday_dates.csv')
    #mon_dates['monday_date'] = pd.to_datetime(mon_dates['monday_date'], errors='coerce')
    filtered_data = mon_dates.merge(data_df, how='left', left_on='monday_date', right_on=date_column)
    filtered_data.drop_duplicates(inplace=True, subset=['monday_date'])
    filtered_data.sort_values(by='monday_date', inplace=True)
    filtered_data.to_csv(type+'.csv')
    filtered_data.info()
    
    return filtered_data

In [249]:
#FX Rate
fx_rate = pd.read_csv('../Data/FxRate.csv')
fx_rate['date'] = fx_rate['Date'].apply(lambda x: str(parser.parse(x))[0:10])
fx_rate.drop(columns=['Date'], inplace=True)
fx_rate = fx_rate.add_suffix('_fxrate')
fx_rate.sort_values(by='date_fxrate', inplace=True)
fx_rate.head(10)

,Price_fxrate,Open_fxrate,High_fxrate,Low_fxrate,Volume_fxrate,Chg%_fxrate,date_fxrate
4076,45.075,45.105,45.145,45.062,0,0.18%,2006-01-02
4075,44.965,45.125,45.145,44.965,0,-0.24%,2006-01-03
4074,44.705,44.885,44.900,44.665,0,-0.58%,2006-01-04
4073,44.600,44.650,44.765,44.600,0,-0.23%,2006-01-05
4072,44.320,44.570,44.785,44.310,0,-0.63%,2006-01-06
4071,44.250,44.320,44.375,44.220,0,-0.16%,2006-01-09
4070,44.185,44.205,44.460,44.165,0,-0.15%,2006-01-10
4069,43.915,44.145,44.145,43.915,0,-0.61%,2006-01-11
4068,44.020,43.950,44.230,43.930,0,0.24%,2006-01-12
4067,44.100,44.060,44.335,44.010,0,0.18%,2006-01-13


In [250]:
filtered_fx_rate = filter_weekly_data(fx_rate, 'fxrate', 'date_fxrate')
filtered_fx_rate.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     783 non-null    int64  
 1   monday_date    783 non-null    object 
 2   Price_fxrate   783 non-null    float64
 3   Open_fxrate    783 non-null    float64
 4   High_fxrate    783 non-null    float64
 5   Low_fxrate     783 non-null    float64
 6   Volume_fxrate  783 non-null    object 
 7   Chg%_fxrate    783 non-null    object 
 8   date_fxrate    783 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Price_fxrate,Open_fxrate,High_fxrate,Low_fxrate,Volume_fxrate,Chg%_fxrate,date_fxrate
0,0,2006-01-02,45.075,45.105,45.145,45.062,0,0.18%,2006-01-02
1,1,2006-01-09,44.250,44.320,44.375,44.220,0,-0.16%,2006-01-09
2,2,2006-01-16,44.170,44.030,44.237,44.010,0,0.16%,2006-01-16
3,3,2006-01-23,44.095,44.125,44.235,44.025,0,-0.17%,2006-01-23
4,4,2006-01-30,44.045,44.245,44.285,44.045,0,0.01%,2006-01-30
5,5,2006-02-06,44.135,44.105,44.257,44.065,0,0.03%,2006-02-06
6,6,2006-02-13,44.180,44.070,44.317,44.010,0,0.17%,2006-02-13
7,7,2006-02-20,44.340,44.260,44.465,44.220,0,0.09%,2006-02-20
8,8,2006-02-27,44.430,44.270,44.510,44.240,0,0.29%,2006-02-27
9,9,2006-03-06,44.270,44.180,44.390,44.170,0,0.15%,2006-03-06


In [251]:
#Rhodium Price
rhodium = pd.read_csv('../Data/Rhodium.csv')
rhodium = rhodium[['date', 'RH']]
rhodium['date'] = rhodium['date'].apply(lambda x: str(parser.parse(x))[0:10])
rhodium = rhodium.add_suffix('_rh')
rhodium.sort_values(by='date_rh', inplace=True)
rhodium.head(10)

,date_rh,RH_rh
0,2006-01-02,3000
1,2006-01-03,3000
2,2006-01-04,3000
3,2006-01-05,3000
4,2006-01-06,3000
5,2006-01-09,3000
6,2006-01-10,3000
7,2006-01-11,3000
8,2006-01-12,3050
9,2006-01-13,3075


In [252]:
rhodium = fill_gapes(rhodium, date_column='date_rh')
rhodium.to_csv('rh_data_expanded.csv')
rhodium.sort_values(by='date_rh', inplace=True)
rhodium.head(10)

Date Column:  date_rh
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,date_rh,RH_rh
0,2006-01-02,3000
1,2006-01-03,3000
2,2006-01-04,3000
3,2006-01-05,3000
4,2006-01-06,3000
4,2006-01-07,3000
4,2006-01-08,3000
5,2006-01-09,3000
6,2006-01-10,3000
7,2006-01-11,3000


In [253]:
rhodium.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5708 entries, 0 to 4049
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date_rh  5708 non-null   object
 1   RH_rh    5708 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.8+ KB


In [254]:
filtered_rhodium_data = filter_weekly_data(rhodium, 'rh', 'date_rh')
temp_df = filtered_rhodium_data.merge(filtered_fx_rate[['monday_date', 'Price_fxrate']], on='monday_date')
temp_df['rh_price_inr'] = temp_df['RH_rh'] * temp_df['Price_fxrate']
filtered_rhodium_data = temp_df[['monday_date', 'date_rh', 'rh_price_inr']]
filtered_rhodium_data.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   783 non-null    int64 
 1   monday_date  783 non-null    object
 2   date_rh      783 non-null    object
 3   RH_rh        783 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.6+ KB


,monday_date,date_rh,rh_price_inr
0,2006-01-02,2006-01-02,135225.00
1,2006-01-09,2006-01-09,132750.00
2,2006-01-16,2006-01-16,136485.30
3,2006-01-23,2006-01-23,138458.30
4,2006-01-30,2006-01-30,147550.75


In [255]:
#Palladium Price
palladium = pd.read_csv('../Data/Palladium.csv')
palladium = palladium[['date', 'PD']]
palladium['date'] = palladium['date'].apply(lambda x: str(parser.parse(x))[0:10])
palladium = palladium.add_suffix('_pd')
palladium.sort_values(by='date_pd', inplace=True)
palladium.head(10)

,date_pd,PD_pd
0,2006-01-02,262
1,2006-01-03,262
2,2006-01-04,272
3,2006-01-05,272
4,2006-01-06,262
5,2006-01-09,274
6,2006-01-10,278
7,2006-01-11,274
8,2006-01-12,274
9,2006-01-13,273


In [256]:
palladium = fill_gapes(palladium, date_column='date_pd')
palladium.to_csv('pd_data_expanded.csv')
palladium.sort_values(by='date_pd', inplace=True)
palladium.head(10)

Date Column:  date_pd
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,date_pd,PD_pd
0,2006-01-02,262
1,2006-01-03,262
2,2006-01-04,272
3,2006-01-05,272
4,2006-01-06,262
4,2006-01-07,262
4,2006-01-08,262
5,2006-01-09,274
6,2006-01-10,278
7,2006-01-11,274


In [257]:
filtered_palladium_data = filter_weekly_data(palladium, 'pd', 'date_pd')
temp_df = filtered_palladium_data.merge(filtered_fx_rate[['monday_date', 'Price_fxrate']], on='monday_date')
temp_df['pd_price_inr'] = temp_df['PD_pd'] * temp_df['Price_fxrate']
filtered_palladium_data = temp_df[['monday_date', 'date_pd', 'pd_price_inr']]
filtered_palladium_data.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   783 non-null    int64 
 1   monday_date  783 non-null    object
 2   date_pd      783 non-null    object
 3   PD_pd        783 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.6+ KB


,monday_date,date_pd,pd_price_inr
0,2006-01-02,2006-01-02,11809.650
1,2006-01-09,2006-01-09,12124.500
2,2006-01-16,2006-01-16,12588.450
3,2006-01-23,2006-01-23,12214.315
4,2006-01-30,2006-01-30,12200.465


In [258]:
inflation = pd.read_csv('../Data/inflation_rate_india.csv')

inflation['date'] = inflation['date'].apply(lambda x: parser.parse(x))
inflation['year'] = inflation['date'].apply(lambda x: str(x)[0:4])
inflation['date'] = inflation['year'].apply(lambda x: find_year_dates(int(x)))
inflation = inflation.explode('date')
inflation = inflation.add_suffix('_inflation')
inflation.sort_values(by='date_inflation', inplace=True)
inflation.head(15)

,date_inflation,Inflation Rate (%)_inflation,Annual Change_inflation,year_inflation
0,1960-01-01,1.7799,NaN,1960
0,1960-01-02,1.7799,NaN,1960
0,1960-01-03,1.7799,NaN,1960
0,1960-01-04,1.7799,NaN,1960
0,1960-01-05,1.7799,NaN,1960
0,1960-01-06,1.7799,NaN,1960
0,1960-01-07,1.7799,NaN,1960
0,1960-01-08,1.7799,NaN,1960
0,1960-01-09,1.7799,NaN,1960
0,1960-01-10,1.7799,NaN,1960


In [259]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22281 entries, 0 to 60
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date_inflation                 22281 non-null  object 
 1    Inflation Rate (%)_inflation  22281 non-null  float64
 2    Annual Change_inflation       21915 non-null  float64
 3   year_inflation                 22281 non-null  object 
dtypes: float64(2), object(2)
memory usage: 870.4+ KB


In [260]:
filtered_inflation_data = filter_weekly_data(inflation, 'inflation', 'date_inflation')
filtered_inflation_data.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     783 non-null    int64  
 1   monday_date                    783 non-null    object 
 2   date_inflation                 783 non-null    object 
 3    Inflation Rate (%)_inflation  783 non-null    float64
 4    Annual Change_inflation       783 non-null    float64
 5   year_inflation                 783 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 42.8+ KB


,Unnamed: 0,monday_date,date_inflation,Inflation Rate (%)_inflation,Annual Change_inflation,year_inflation
0,0,2006-01-02,2006-01-02,5.7965,1.55,2006
1,1,2006-01-09,2006-01-09,5.7965,1.55,2006
2,2,2006-01-16,2006-01-16,5.7965,1.55,2006
3,3,2006-01-23,2006-01-23,5.7965,1.55,2006
4,4,2006-01-30,2006-01-30,5.7965,1.55,2006
5,5,2006-02-06,2006-02-06,5.7965,1.55,2006
6,6,2006-02-13,2006-02-13,5.7965,1.55,2006
7,7,2006-02-20,2006-02-20,5.7965,1.55,2006
8,8,2006-02-27,2006-02-27,5.7965,1.55,2006
9,9,2006-03-06,2006-03-06,5.7965,1.55,2006


In [261]:
# GDP Per Capita
gdp = pd.read_csv('../Data/india-gdp-per-capita.csv')

gdp['date'] = gdp['date'].apply(lambda x: parser.parse(x))
gdp['year'] = gdp['date'].apply(lambda x: str(x)[0:4])
gdp['date'] = gdp['year'].apply(lambda x: find_year_dates(int(x)))
gdp = gdp.explode('date')
gdp = gdp.add_suffix('_gdp')
gdp.sort_values(by='date_gdp', inplace=True)
gdp.head(15)

,date_gdp,GDP Per Capita (US $)_gdp,year_gdp
0,1960-01-01,82.1886,1960
0,1960-01-02,82.1886,1960
0,1960-01-03,82.1886,1960
0,1960-01-04,82.1886,1960
0,1960-01-05,82.1886,1960
0,1960-01-06,82.1886,1960
0,1960-01-07,82.1886,1960
0,1960-01-08,82.1886,1960
0,1960-01-09,82.1886,1960
0,1960-01-10,82.1886,1960


In [262]:
filtered_gdp_data = filter_weekly_data(gdp, 'gdp', 'date_gdp')

temp_df = filtered_gdp_data.merge(filtered_fx_rate[['monday_date', 'Price_fxrate']], on='monday_date')
temp_df['gdp_per_capita'] = temp_df['GDP Per Capita (US $)_gdp'] * temp_df['Price_fxrate']
filtered_gdp_data = temp_df[['monday_date', 'date_gdp', 'gdp_per_capita']]
filtered_gdp_data.head(5)


filtered_gdp_data.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 783 non-null    int64  
 1   monday_date                783 non-null    object 
 2   date_gdp                   783 non-null    object 
 3   GDP Per Capita (US $)_gdp  783 non-null    float64
 4   year_gdp                   783 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 36.7+ KB


,monday_date,date_gdp,gdp_per_capita
0,2006-01-02,2006-01-02,36364.404998
1,2006-01-09,2006-01-09,35698.833525
2,2006-01-16,2006-01-16,35634.293261
3,2006-01-23,2006-01-23,35573.786764
4,2006-01-30,2006-01-30,35533.449099
5,2006-02-06,2006-02-06,35606.056895
6,2006-02-13,2006-02-13,35642.360794
7,2006-02-20,2006-02-20,35771.441322
8,2006-02-27,2006-02-27,35844.049119
9,2006-03-06,2006-03-06,35714.968591


In [263]:
# GNI Per Capita
gni = pd.read_csv('../Data/india-gni-per-capita.csv')

gni['date'] = gni['date'].apply(lambda x: parser.parse(x))
gni['year'] = gni['date'].apply(lambda x: str(x)[0:4])
gni['date'] = gni['year'].apply(lambda x: find_year_dates(int(x)))
gni = gni.explode('date')
gni = gni.add_suffix('_gni')
gni.sort_values(by='date_gni', inplace=True)

filtered_gni_data = filter_weekly_data(gni, 'gni', 'date_gni')

temp_df = filtered_gni_data.merge(filtered_fx_rate[['monday_date', 'Price_fxrate']], on='monday_date')
temp_df['gni_per_capita'] = temp_df['GNI Per Capita (US $)_gni'] * temp_df['Price_fxrate']
filtered_gni_data = temp_df[['monday_date', 'date_gni', 'gni_per_capita']]
filtered_gni_data.head(5)


filtered_gni_data.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 783 non-null    int64 
 1   monday_date                783 non-null    object
 2   date_gni                   783 non-null    object
 3   GNI Per Capita (US $)_gni  783 non-null    int64 
 4   year_gni                   783 non-null    object
dtypes: int64(2), object(3)
memory usage: 36.7+ KB


,monday_date,date_gni,gni_per_capita
0,2006-01-02,2006-01-02,35609.25
1,2006-01-09,2006-01-09,34957.50
2,2006-01-16,2006-01-16,34894.30
3,2006-01-23,2006-01-23,34835.05
4,2006-01-30,2006-01-30,34795.55
5,2006-02-06,2006-02-06,34866.65
6,2006-02-13,2006-02-13,34902.20
7,2006-02-20,2006-02-20,35028.60
8,2006-02-27,2006-02-27,35099.70
9,2006-03-06,2006-03-06,34973.30


In [264]:
# 10Yr Bond Yield
bondYield = pd.read_csv('../Data/India 10-Year Bond Yield.csv')
bondYield['date'] = bondYield['Date'].apply(lambda x: str(parser.parse(x))[0:10])
bondYield.drop(columns=['Date'], inplace=True)
bondYield = bondYield.add_suffix('_10YrBondRate')
bondYield.sort_values(by='date_10YrBondRate', inplace=True)
bondYield = fill_gapes(bondYield, date_column='date_10YrBondRate')
bondYield.to_csv('10YrBondRate_data_expanded.csv')
bondYield.sort_values(by='date_10YrBondRate', inplace=True)
bondYield.head(10)

filtered_10YrBondRate = filter_weekly_data(bondYield, '10YrBondRate', 'date_10YrBondRate')
filtered_10YrBondRate.head(15)

Date Column:  date_10YrBondRate
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             783 non-null    int64  
 1   monday_date            783 non-null    object 
 2   Price_10YrBondRate     783 non-null    float64
 3   Open_10YrBondRate      783 non-null    float64
 4   High_10YrBondRate      783 non-null    float64
 5   Low_10YrBondRate       783 non-null    float64
 6   Change %_10YrBondRate  783 non-null    object 
 7   date_10YrBondRate      783 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 55.1+ KB


,Unnamed: 0,monday_date,Price_10YrBondRate,Open_10YrBondRate,High_10YrBondRate,Low_10YrBondRate,Change %_10YrBondRate,date_10YrBondRate
0,0,2006-01-02,7.180,7.180,7.180,7.180,0.98%,2006-01-02
1,1,2006-01-09,7.187,7.187,7.187,7.187,0.06%,2006-01-09
2,2,2006-01-16,7.202,7.202,7.202,7.202,-0.15%,2006-01-16
3,3,2006-01-23,7.205,7.205,7.205,7.205,0.03%,2006-01-23
4,4,2006-01-30,7.425,7.425,7.425,7.425,-0.80%,2006-01-30
5,5,2006-02-06,7.326,7.326,7.326,7.326,-0.31%,2006-02-06
6,6,2006-02-13,7.345,7.345,7.345,7.345,-0.10%,2006-02-13
7,7,2006-02-20,7.348,7.348,7.348,7.348,0.08%,2006-02-20
8,8,2006-02-27,7.371,7.371,7.371,7.371,-0.24%,2006-02-27
9,9,2006-03-06,7.477,7.477,7.477,7.477,0.40%,2006-03-06


In [265]:
# Interest Rate

In [266]:
interest_rate = pd.read_csv('../Data/interest_rate_india.csv')
interest_rate['date'] = interest_rate['Year'].apply(lambda x: find_all_financial_year_dates(x))
interest_rate = interest_rate.explode('date')
interest_rate = interest_rate.add_suffix('_interest')
interest_rate.sort_values(by='date_interest', inplace=True)
interest_rate.head(15)

,Year_interest,1 to 3 yrs._interest,Over 3 yrs. & upto 5 yrs._interest,Above 5 yrs._interest,date_interest
0,1975-76,8,9,10,1975-04-01
0,1975-76,8,9,10,1975-04-02
0,1975-76,8,9,10,1975-04-03
0,1975-76,8,9,10,1975-04-04
0,1975-76,8,9,10,1975-04-05
0,1975-76,8,9,10,1975-04-06
0,1975-76,8,9,10,1975-04-07
0,1975-76,8,9,10,1975-04-08
0,1975-76,8,9,10,1975-04-09
0,1975-76,8,9,10,1975-04-10


In [267]:
filtered_interest_rate = filter_weekly_data(interest_rate, 'interest', 'date_interest')
filtered_interest_rate.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Unnamed: 0                          783 non-null    int64 
 1   monday_date                         783 non-null    object
 2   Year_interest                       783 non-null    object
 3   1 to 3 yrs._interest                783 non-null    object
 4   Over 3 yrs. & upto 5 yrs._interest  783 non-null    object
 5   Above 5 yrs._interest               783 non-null    object
 6   date_interest                       783 non-null    object
dtypes: int64(1), object(6)
memory usage: 48.9+ KB


,Unnamed: 0,monday_date,Year_interest,1 to 3 yrs._interest,Over 3 yrs. & upto 5 yrs._interest,Above 5 yrs._interest,date_interest
0,0,2006-01-02,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-01-02
1,1,2006-01-09,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-01-09
2,2,2006-01-16,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-01-16
3,3,2006-01-23,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-01-23
4,4,2006-01-30,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-01-30
5,5,2006-02-06,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-02-06
6,6,2006-02-13,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-02-13
7,7,2006-02-20,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-02-20
8,8,2006-02-27,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-02-27
9,9,2006-03-06,2005-06,6.00-6.50,6.25-7.00,6.25-7.00,2006-03-06


In [268]:
dates = all_dates(start_year,end_year)
print('start_date', dates[0])
print('end_date', dates[len(dates)-1])

start_date 2006-01-01
end_date 2020-12-31


In [269]:
#Uncertainty Index

In [270]:
uncertainty_index = pd.read_csv('../Data/India_Policy_Uncertainty_Data.csv')
uncertainty_index['date'] = uncertainty_index[['Year', 'Month']].apply(lambda data: find_dates(int(data['Year']), int(data['Month'])), axis=1)
uncertainty_index = uncertainty_index.explode('date')
uncertainty_index = uncertainty_index.add_suffix('_uncertainty')
uncertainty_index.sort_values(by='date_uncertainty', inplace=True) 
uncertainty_index.head(10)

,Year_uncertainty,Month_uncertainty,Uncertainty Index_uncertainty,date_uncertainty
0,2003,1,49.281837,2003-01-01
0,2003,1,49.281837,2003-01-02
0,2003,1,49.281837,2003-01-03
0,2003,1,49.281837,2003-01-04
0,2003,1,49.281837,2003-01-05
0,2003,1,49.281837,2003-01-06
0,2003,1,49.281837,2003-01-07
0,2003,1,49.281837,2003-01-08
0,2003,1,49.281837,2003-01-09
0,2003,1,49.281837,2003-01-10


In [271]:
filtered_uncertainty = filter_weekly_data(uncertainty_index, 'uncertainty', 'date_uncertainty')
filtered_uncertainty.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     783 non-null    int64  
 1   monday_date                    783 non-null    object 
 2   Year_uncertainty               783 non-null    int64  
 3   Month_uncertainty              783 non-null    int64  
 4   Uncertainty Index_uncertainty  783 non-null    float64
 5   date_uncertainty               783 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 42.8+ KB


,Unnamed: 0,monday_date,Year_uncertainty,Month_uncertainty,Uncertainty Index_uncertainty,date_uncertainty
0,0,2006-01-02,2006,1,39.990150,2006-01-02
1,1,2006-01-09,2006,1,39.990150,2006-01-09
2,2,2006-01-16,2006,1,39.990150,2006-01-16
3,3,2006-01-23,2006,1,39.990150,2006-01-23
4,4,2006-01-30,2006,1,39.990150,2006-01-30
5,5,2006-02-06,2006,2,31.373428,2006-02-06
6,6,2006-02-13,2006,2,31.373428,2006-02-13
7,7,2006-02-20,2006,2,31.373428,2006-02-20
8,8,2006-02-27,2006,2,31.373428,2006-02-27
9,9,2006-03-06,2006,3,32.268181,2006-03-06


In [272]:
# Nifty50
nifty50 = pd.read_csv('../Data/Nifty50.csv')
nifty50['date'] = nifty50['Date'].apply(lambda x: str(parser.parse(x))[0:10])
nifty50.drop(columns=['Date'], inplace=True)
nifty50 = nifty50.add_suffix('_nifty')
nifty50.sort_values(by='date_nifty', inplace=True)
nifty50.head(10)

,Price_nifty,Open_nifty,High_nifty,Low_nifty,Volume_nifty,Chg%_nifty,date_nifty
3869,"2,835.95","2,836.80","2,849.45","2,825.40",51.92M,-0.02%,2006-01-02
3868,"2,883.35","2,835.95","2,887.20","2,832.05",56.92M,1.67%,2006-01-03
3867,"2,904.40","2,883.00","2,909.35","2,883.00",82.02M,0.73%,2006-01-04
3866,"2,899.85","2,904.45","2,916.20","2,884.80",70.08M,-0.16%,2006-01-05
3865,"2,914.00","2,899.85","2,921.70","2,877.25",83.35M,0.49%,2006-01-06
3864,"2,910.10","2,913.35","2,927.25","2,898.25",74.82M,-0.13%,2006-01-09
3863,"2,870.80","2,910.15","2,913.05","2,865.60",71.12M,-1.35%,2006-01-10
3862,"2,850.70","2,869.35","2,869.35","2,824.00",92.18M,-0.70%,2006-01-12
3861,"2,850.55","2,851.40","2,878.35","2,846.70",66.17M,-0.01%,2006-01-13
3860,"2,833.10","2,851.35","2,855.70","2,824.05",57.83M,-0.61%,2006-01-16


In [273]:
nifty50 = fill_gapes(nifty50, date_column='date_nifty')
nifty50.to_csv('nifty_data_expanded.csv')
nifty50.sort_values(by='date_nifty', inplace=True)
nifty50.head(10)

Date Column:  date_nifty
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Price_nifty,Open_nifty,High_nifty,Low_nifty,Volume_nifty,Chg%_nifty,date_nifty
3869,"2,835.95","2,836.80","2,849.45","2,825.40",51.92M,-0.02%,2006-01-02
3868,"2,883.35","2,835.95","2,887.20","2,832.05",56.92M,1.67%,2006-01-03
3867,"2,904.40","2,883.00","2,909.35","2,883.00",82.02M,0.73%,2006-01-04
3866,"2,899.85","2,904.45","2,916.20","2,884.80",70.08M,-0.16%,2006-01-05
3865,"2,914.00","2,899.85","2,921.70","2,877.25",83.35M,0.49%,2006-01-06
3865,"2,914.00","2,899.85","2,921.70","2,877.25",83.35M,0.49%,2006-01-07
3865,"2,914.00","2,899.85","2,921.70","2,877.25",83.35M,0.49%,2006-01-08
3864,"2,910.10","2,913.35","2,927.25","2,898.25",74.82M,-0.13%,2006-01-09
3863,"2,870.80","2,910.15","2,913.05","2,865.60",71.12M,-1.35%,2006-01-10
3863,"2,870.80","2,910.15","2,913.05","2,865.60",71.12M,-1.35%,2006-01-11


In [274]:
filtered_nifty50 = filter_weekly_data(nifty50, 'nifty', 'date_nifty')
filtered_nifty50.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    783 non-null    int64 
 1   monday_date   783 non-null    object
 2   Price_nifty   783 non-null    object
 3   Open_nifty    783 non-null    object
 4   High_nifty    783 non-null    object
 5   Low_nifty     783 non-null    object
 6   Volume_nifty  783 non-null    object
 7   Chg%_nifty    783 non-null    object
 8   date_nifty    783 non-null    object
dtypes: int64(1), object(8)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Price_nifty,Open_nifty,High_nifty,Low_nifty,Volume_nifty,Chg%_nifty,date_nifty
0,0,2006-01-02,"2,835.95","2,836.80","2,849.45","2,825.40",51.92M,-0.02%,2006-01-02
1,1,2006-01-09,"2,910.10","2,913.35","2,927.25","2,898.25",74.82M,-0.13%,2006-01-09
2,2,2006-01-16,"2,833.10","2,851.35","2,855.70","2,824.05",57.83M,-0.61%,2006-01-16
3,3,2006-01-23,"2,884.05","2,900.30","2,900.30","2,870.95",65.13M,-0.58%,2006-01-23
4,4,2006-01-30,"2,974.50","2,983.30","3,002.20","2,963.65",99.40M,-0.28%,2006-01-30
5,5,2006-02-06,"3,000.45","2,940.95","3,009.45","2,928.10",70.48M,2.04%,2006-02-06
6,6,2006-02-13,"3,041.15","3,026.15","3,042.75","3,018.50",66.17M,0.45%,2006-02-13
7,7,2006-02-20,"3,005.85","2,982.35","3,010.65","2,955.85",61.70M,0.82%,2006-02-20
8,8,2006-02-27,"3,067.45","3,050.30","3,070.35","3,050.30",53.58M,0.57%,2006-02-27
9,9,2006-03-06,"3,190.40","3,147.25","3,194.00","3,147.20",70.23M,1.37%,2006-03-06


In [275]:
# S&P500
snp500 = pd.read_csv('../Data/S&P500.csv')
snp500['date'] = snp500['Date'].apply(lambda x: str(parser.parse(x))[0:10])
snp500.drop(columns=['Date'], inplace=True)
snp500 = snp500.add_suffix('_snp')
snp500.sort_values(by='date_snp', inplace=True)
snp500.head(10)

,Price_snp,Open_snp,High_snp,Low_snp,Volume_snp,Chg%_snp,date_snp
3931,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-01
3930,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-02
3929,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-03
3928,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-04
3927,"1,273.48","1,273.46","1,276.91","1,270.30",0,0.00%,2006-01-05
3926,"1,285.45","1,273.48","1,286.09","1,273.48",0,0.94%,2006-01-06
3925,"1,290.15","1,285.45","1,290.78","1,284.82",0,0.37%,2006-01-09
3924,"1,289.69","1,290.15","1,290.15","1,283.76",0,-0.04%,2006-01-10
3923,"1,294.18","1,289.72","1,294.90","1,288.12",0,0.35%,2006-01-11
3922,"1,286.06","1,294.18","1,294.18","1,285.04",0,-0.63%,2006-01-12


In [276]:
snp500 = fill_gapes(snp500, date_column='date_snp')
snp500.to_csv('snp_data_expanded.csv')
snp500.sort_values(by='date_snp', inplace=True)
snp500.head(10)

Date Column:  date_snp
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Price_snp,Open_snp,High_snp,Low_snp,Volume_snp,Chg%_snp,date_snp
3931,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-01
3930,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-02
3929,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-03
3928,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-04
3927,"1,273.48","1,273.46","1,276.91","1,270.30",0,0.00%,2006-01-05
3926,"1,285.45","1,273.48","1,286.09","1,273.48",0,0.94%,2006-01-06
3926,"1,285.45","1,273.48","1,286.09","1,273.48",0,0.94%,2006-01-07
3926,"1,285.45","1,273.48","1,286.09","1,273.48",0,0.94%,2006-01-08
3925,"1,290.15","1,285.45","1,290.78","1,284.82",0,0.37%,2006-01-09
3924,"1,289.69","1,290.15","1,290.15","1,283.76",0,-0.04%,2006-01-10


In [277]:
filtered_snp500 = filter_weekly_data(snp500, 'snp', 'date_snp')
filtered_snp500.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   783 non-null    int64 
 1   monday_date  783 non-null    object
 2   Price_snp    783 non-null    object
 3   Open_snp     783 non-null    object
 4   High_snp     783 non-null    object
 5   Low_snp      783 non-null    object
 6   Volume_snp   783 non-null    int64 
 7   Chg%_snp     783 non-null    object
 8   date_snp     783 non-null    object
dtypes: int64(2), object(7)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Price_snp,Open_snp,High_snp,Low_snp,Volume_snp,Chg%_snp,date_snp
0,0,2006-01-02,"1,273.46","1,268.80","1,275.37","1,267.74",0,0.37%,2006-01-02
1,1,2006-01-09,"1,290.15","1,285.45","1,290.78","1,284.82",0,0.37%,2006-01-09
2,2,2006-01-16,"1,287.61","1,286.06","1,288.96","1,282.78",0,0.12%,2006-01-16
3,3,2006-01-23,"1,263.82","1,261.49","1,268.19","1,261.49",0,0.18%,2006-01-23
4,4,2006-01-30,"1,285.19","1,283.72","1,287.94","1,283.51",0,0.11%,2006-01-30
5,5,2006-02-06,"1,265.02","1,264.03","1,267.04","1,261.62",0,0.08%,2006-02-06
6,6,2006-02-13,"1,262.86","1,266.99","1,266.99","1,258.34",0,-0.33%,2006-02-13
7,7,2006-02-20,"1,287.24","1,289.38","1,289.47","1,284.07",0,-0.17%,2006-02-20
8,8,2006-02-27,"1,294.12","1,289.43","1,297.57","1,289.43",0,0.36%,2006-02-27
9,9,2006-03-06,"1,278.26","1,287.23","1,288.23","1,275.67",0,-0.70%,2006-03-06


In [278]:
# WPM
wpm = pd.read_csv('../Data/WPM.csv')
wpm['date'] = wpm['Date'].apply(lambda x: str(parser.parse(x))[0:10])
wpm.drop(columns=['Date'], inplace=True)
wpm = wpm.add_suffix('_wpm')
wpm.sort_values(by='date_wpm', inplace=True)
wpm.head(10)

,Open_wpm,High_wpm,Low_wpm,Close_wpm,Adj Close_wpm,Volume_wpm,date_wpm
0,3.10,3.17,3.07,3.12,2.715731,343200,2005-07-06
1,3.20,3.20,3.15,3.17,2.759253,132500,2005-07-07
2,3.20,3.23,3.17,3.19,2.776660,162200,2005-07-08
3,3.22,3.24,3.19,3.24,2.820183,208200,2005-07-11
4,3.27,3.29,3.25,3.29,2.863703,203800,2005-07-12
5,3.28,3.29,3.25,3.25,2.828886,93200,2005-07-13
6,3.23,3.28,3.23,3.24,2.820183,82000,2005-07-14
7,3.30,3.30,3.20,3.22,2.802774,223500,2005-07-15
8,3.25,3.25,3.16,3.19,2.776660,68300,2005-07-18
9,3.19,3.21,3.16,3.19,2.776660,203600,2005-07-19


In [279]:
wpm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4057 entries, 0 to 4056
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open_wpm       4057 non-null   float64
 1   High_wpm       4057 non-null   float64
 2   Low_wpm        4057 non-null   float64
 3   Close_wpm      4057 non-null   float64
 4   Adj Close_wpm  4057 non-null   float64
 5   Volume_wpm     4057 non-null   int64  
 6   date_wpm       4057 non-null   object 
dtypes: float64(5), int64(1), object(1)
memory usage: 253.6+ KB


In [280]:
wpm = fill_gapes(wpm, date_column='date_wpm', start_year=2005)
wpm.to_csv('wpm_data_expanded.csv')
wpm.sort_values(by='date_wpm', inplace=True)
wpm.head(10)
#Gap filling is configured to happen since 2006

Date Column:  date_wpm
Start Year:  2005
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Open_wpm,High_wpm,Low_wpm,Close_wpm,Adj Close_wpm,Volume_wpm,date_wpm
0,3.10,3.17,3.07,3.12,2.715731,343200,2005-07-06
1,3.20,3.20,3.15,3.17,2.759253,132500,2005-07-07
2,3.20,3.23,3.17,3.19,2.776660,162200,2005-07-08
2,3.20,3.23,3.17,3.19,2.776660,162200,2005-07-09
2,3.20,3.23,3.17,3.19,2.776660,162200,2005-07-10
3,3.22,3.24,3.19,3.24,2.820183,208200,2005-07-11
4,3.27,3.29,3.25,3.29,2.863703,203800,2005-07-12
5,3.28,3.29,3.25,3.25,2.828886,93200,2005-07-13
6,3.23,3.28,3.23,3.24,2.820183,82000,2005-07-14
7,3.30,3.30,3.20,3.22,2.802774,223500,2005-07-15


In [281]:
filtered_wpm = filter_weekly_data(wpm, 'wpm', 'date_wpm')
filtered_wpm.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     783 non-null    int64  
 1   monday_date    783 non-null    object 
 2   Open_wpm       783 non-null    float64
 3   High_wpm       783 non-null    float64
 4   Low_wpm        783 non-null    float64
 5   Close_wpm      783 non-null    float64
 6   Adj Close_wpm  783 non-null    float64
 7   Volume_wpm     783 non-null    int64  
 8   date_wpm       783 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Open_wpm,High_wpm,Low_wpm,Close_wpm,Adj Close_wpm,Volume_wpm,date_wpm
0,0,2006-01-02,5.65,5.83,5.61,5.80,5.048476,608500,2006-01-02
1,1,2006-01-09,6.10,6.10,5.90,5.93,5.161630,852900,2006-01-09
2,2,2006-01-16,5.92,6.06,5.85,6.05,5.266082,782000,2006-01-16
3,3,2006-01-23,5.75,5.83,5.71,5.78,5.031067,361900,2006-01-23
4,4,2006-01-30,7.22,7.56,7.17,7.45,6.484680,1598300,2006-01-30
5,5,2006-02-06,7.69,7.88,7.61,7.63,6.641355,851900,2006-02-06
6,6,2006-02-13,6.74,6.85,6.35,6.42,5.588140,1264100,2006-02-13
7,7,2006-02-20,7.27,7.51,7.27,7.33,6.380228,1063000,2006-02-20
8,8,2006-02-27,8.28,8.29,7.91,7.94,6.911189,1562900,2006-02-27
9,9,2006-03-06,9.50,9.69,9.05,9.18,7.990518,2119100,2006-03-06


In [282]:
# EGO
ego = pd.read_csv('../Data/EGO.csv')
ego['date'] = ego['Date'].apply(lambda x: str(parser.parse(x))[0:10])
ego.drop(columns=['Date'], inplace=True)
ego = ego.add_suffix('_ego')
ego.sort_values(by='date_ego', inplace=True)
ego.head(10)

,Open_ego,High_ego,Low_ego,Close_ego,Adj Close_ego,Volume_ego,date_ego
0,8.05,8.25,7.55,7.60,7.275973,129580,2003-01-23
1,7.75,7.85,7.35,7.60,7.275973,73140,2003-01-24
2,7.85,7.95,7.55,7.65,7.323843,35620,2003-01-27
3,7.60,7.75,7.40,7.70,7.371710,15000,2003-01-28
4,7.75,7.75,6.95,7.30,6.988765,44600,2003-01-29
5,7.00,7.50,7.00,7.35,7.036633,18160,2003-01-30
6,7.80,7.80,7.30,7.50,7.180238,11320,2003-01-31
7,7.40,7.65,7.35,7.60,7.275973,16580,2003-02-03
8,7.70,8.15,7.70,8.00,7.658920,35040,2003-02-04
9,8.00,8.25,7.60,7.60,7.275973,28560,2003-02-05


In [283]:
ego = fill_gapes(ego, date_column='date_ego', start_year=2003)
ego.to_csv('ego_data_expanded.csv')
ego.sort_values(by='date_ego', inplace=True)
ego.head(10)

Date Column:  date_ego
Start Year:  2003
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Open_ego,High_ego,Low_ego,Close_ego,Adj Close_ego,Volume_ego,date_ego
0,8.05,8.25,7.55,7.60,7.275973,129580,2003-01-23
1,7.75,7.85,7.35,7.60,7.275973,73140,2003-01-24
1,7.75,7.85,7.35,7.60,7.275973,73140,2003-01-25
1,7.75,7.85,7.35,7.60,7.275973,73140,2003-01-26
2,7.85,7.95,7.55,7.65,7.323843,35620,2003-01-27
3,7.60,7.75,7.40,7.70,7.371710,15000,2003-01-28
4,7.75,7.75,6.95,7.30,6.988765,44600,2003-01-29
5,7.00,7.50,7.00,7.35,7.036633,18160,2003-01-30
6,7.80,7.80,7.30,7.50,7.180238,11320,2003-01-31
6,7.80,7.80,7.30,7.50,7.180238,11320,2003-02-01


In [284]:
filtered_ego = filter_weekly_data(ego, 'ego', 'date_ego')
filtered_ego.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     783 non-null    int64  
 1   monday_date    783 non-null    object 
 2   Open_ego       783 non-null    float64
 3   High_ego       783 non-null    float64
 4   Low_ego        783 non-null    float64
 5   Close_ego      783 non-null    float64
 6   Adj Close_ego  783 non-null    float64
 7   Volume_ego     783 non-null    int64  
 8   date_ego       783 non-null    object 
dtypes: float64(5), int64(2), object(2)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Open_ego,High_ego,Low_ego,Close_ego,Adj Close_ego,Volume_ego,date_ego
0,0,2006-01-02,24.850000,24.850000,24.049999,24.500000,23.455441,148440,2006-01-02
1,1,2006-01-09,26.850000,27.200001,26.000000,26.650000,25.513779,262860,2006-01-09
2,2,2006-01-16,25.600000,26.200001,25.250000,25.950001,24.843624,220860,2006-01-16
3,3,2006-01-23,23.150000,23.350000,22.799999,23.299999,22.306606,320520,2006-01-23
4,4,2006-01-30,24.600000,25.299999,24.450001,24.850000,23.790525,309160,2006-01-30
5,5,2006-02-06,24.400000,24.900000,24.049999,24.200001,23.168232,190880,2006-02-06
6,6,2006-02-13,21.150000,21.650000,20.049999,20.350000,19.482376,290000,2006-02-13
7,7,2006-02-20,23.450001,23.799999,22.600000,22.799999,21.827919,252000,2006-02-20
8,8,2006-02-27,22.000000,22.000000,20.950001,21.299999,20.391874,224320,2006-02-27
9,9,2006-03-06,23.000000,23.000000,21.049999,21.750000,20.822693,272800,2006-03-06


In [285]:
# Gold Price
gold_price_df = combine_data('../Data/GoldPrice')
gold_price_df['date']=gold_price_df['Date'].apply(lambda x: str(x)[0:10])
gold_price_df.drop(columns=['Date'], inplace=True)
gold_price_df = gold_price_df.add_suffix('_gold')
gold_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4213 entries, 0 to 255
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Commodity_gold        4213 non-null   object
 1   Unit_gold             4213 non-null   object
 2   Location_gold         4213 non-null   object
 3   Spot Price(Rs.)_gold  4213 non-null   int64 
 4   Up/Down_gold          4213 non-null   object
 5   Time_gold             4213 non-null   object
 6   date_gold             4213 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.3+ KB


In [286]:
gold_price_df.sort_values(by='date_gold', inplace=True)
gold_price_df.head(10)

,Commodity_gold,Unit_gold,Location_gold,Spot Price(Rs.)_gold,Up/Down_gold,Time_gold,date_gold
290,GOLD,10 GRMS,AHMEDABAD,7640,+,12:37:00,2006-01-02
289,GOLD,10 GRMS,AHMEDABAD,7835,+,16:47:00,2006-01-04
288,GOLD,10 GRMS,AHMEDABAD,7760,-,12:13:00,2006-01-05
287,GOLD,10 GRMS,AHMEDABAD,7750,+,16:43:00,2006-01-06
286,GOLD,10 GRMS,AHMEDABAD,7870,+,12:33:00,2006-01-07
285,GOLD,10 GRMS,AHMEDABAD,7845,-,16:50:00,2006-01-09
284,GOLD,10 GRMS,AHMEDABAD,7900,-,16:40:00,2006-01-10
283,GOLD,10 GRMS,AHMEDABAD,7885,-,12:39:00,2006-01-11
282,GOLD,10 GRMS,AHMEDABAD,7915,+,16:39:00,2006-01-12
281,GOLD,10 GRMS,AHMEDABAD,7925,+,18:28:00,2006-01-13


In [287]:
gold_price_df = fill_gapes(gold_price_df, date_column='date_gold')
gold_price_df.to_csv('gold_data_expanded.csv')
gold_price_df.sort_values(by='date_gold', inplace=True)
gold_price_df.head(10)

Date Column:  date_gold
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Commodity_gold,Unit_gold,Location_gold,Spot Price(Rs.)_gold,Up/Down_gold,Time_gold,date_gold
290,GOLD,10 GRMS,AHMEDABAD,7640,+,12:37:00,2006-01-02
290,GOLD,10 GRMS,AHMEDABAD,7640,+,12:37:00,2006-01-03
289,GOLD,10 GRMS,AHMEDABAD,7835,+,16:47:00,2006-01-04
288,GOLD,10 GRMS,AHMEDABAD,7760,-,12:13:00,2006-01-05
287,GOLD,10 GRMS,AHMEDABAD,7750,+,16:43:00,2006-01-06
286,GOLD,10 GRMS,AHMEDABAD,7870,+,12:33:00,2006-01-07
286,GOLD,10 GRMS,AHMEDABAD,7870,+,12:33:00,2006-01-08
285,GOLD,10 GRMS,AHMEDABAD,7845,-,16:50:00,2006-01-09
284,GOLD,10 GRMS,AHMEDABAD,7900,-,16:40:00,2006-01-10
283,GOLD,10 GRMS,AHMEDABAD,7885,-,12:39:00,2006-01-11


In [288]:
filtered_gold_price = filter_weekly_data(gold_price_df, 'gold_price', 'date_gold')
filtered_gold_price.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            783 non-null    int64 
 1   monday_date           783 non-null    object
 2   Commodity_gold        783 non-null    object
 3   Unit_gold             783 non-null    object
 4   Location_gold         783 non-null    object
 5   Spot Price(Rs.)_gold  783 non-null    int64 
 6   Up/Down_gold          783 non-null    object
 7   Time_gold             783 non-null    object
 8   date_gold             783 non-null    object
dtypes: int64(2), object(7)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Commodity_gold,Unit_gold,Location_gold,Spot Price(Rs.)_gold,Up/Down_gold,Time_gold,date_gold
0,0,2006-01-02,GOLD,10 GRMS,AHMEDABAD,7640,+,12:37:00,2006-01-02
1,1,2006-01-09,GOLD,10 GRMS,AHMEDABAD,7845,-,16:50:00,2006-01-09
2,2,2006-01-16,GOLD,10 GRMS,AHMEDABAD,8010,=,14:44:00,2006-01-16
3,3,2006-01-23,GOLD,10 GRMS,AHMEDABAD,8030,+,12:53:00,2006-01-23
4,4,2006-01-30,GOLD,10 GRMS,AHMEDABAD,8080,-,16:55:00,2006-01-30
5,5,2006-02-06,GOLD,10 GRMS,AHMEDABAD,8200,-,13:09:00,2006-02-06
6,6,2006-02-13,GOLD,10 GRMS,AHMEDABAD,7930,=,18:28:00,2006-02-13
7,7,2006-02-20,GOLD,10 GRMS,AHMEDABAD,8020,+,12:36:00,2006-02-20
8,8,2006-02-27,GOLD,10 GRMS,AHMEDABAD,8080,=,17:38:00,2006-02-27
9,9,2006-03-06,GOLD,10 GRMS,AHMEDABAD,8225,=,16:47:00,2006-03-06


In [289]:
# Silver Price
silver_price_df = combine_data('../Data/Silver')
silver_price_df['date']=silver_price_df['Date'].apply(lambda x: str(x)[0:10])
silver_price_df.drop(columns=['Date'], inplace=True)
silver_price_df = silver_price_df.add_suffix('_silver')
silver_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4095 entries, 0 to 254
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Commodity_silver        4095 non-null   object
 1   Unit_silver             4095 non-null   object
 2   Location_silver         4095 non-null   object
 3   Spot Price(Rs.)_silver  4095 non-null   int64 
 4   Up/Down_silver          4095 non-null   object
 5   Time_silver             4095 non-null   object
 6   date_silver             4095 non-null   object
dtypes: int64(1), object(6)
memory usage: 255.9+ KB


In [290]:
silver_price_df = fill_gapes(silver_price_df, date_column='date_silver')
silver_price_df.to_csv('silver_data_expanded.csv')
silver_price_df.sort_values(by='date_silver', inplace=True)
silver_price_df.head(10)

Date Column:  date_silver
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Commodity_silver,Unit_silver,Location_silver,Spot Price(Rs.)_silver,Up/Down_silver,Time_silver,date_silver
289,SILVER,1 KGS,AHMEDABAD,13300,=,12:37:00,2006-01-02
289,SILVER,1 KGS,AHMEDABAD,13300,=,12:37:00,2006-01-03
288,SILVER,1 KGS,AHMEDABAD,13750,+,16:47:00,2006-01-04
287,SILVER,1 KGS,AHMEDABAD,13500,-,12:13:00,2006-01-05
286,SILVER,1 KGS,AHMEDABAD,13300,=,16:44:00,2006-01-06
285,SILVER,1 KGS,AHMEDABAD,13650,+,12:35:00,2006-01-07
285,SILVER,1 KGS,AHMEDABAD,13650,+,12:35:00,2006-01-08
284,SILVER,1 KGS,AHMEDABAD,13500,=,16:50:00,2006-01-09
283,SILVER,1 KGS,AHMEDABAD,13450,-,16:35:00,2006-01-10
282,SILVER,1 KGS,AHMEDABAD,13200,=,17:02:00,2006-01-11


In [291]:
filtered_silver_price = filter_weekly_data(silver_price_df, 'silver_price', 'date_silver')
filtered_silver_price.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              783 non-null    int64 
 1   monday_date             783 non-null    object
 2   Commodity_silver        783 non-null    object
 3   Unit_silver             783 non-null    object
 4   Location_silver         783 non-null    object
 5   Spot Price(Rs.)_silver  783 non-null    int64 
 6   Up/Down_silver          783 non-null    object
 7   Time_silver             783 non-null    object
 8   date_silver             783 non-null    object
dtypes: int64(2), object(7)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Commodity_silver,Unit_silver,Location_silver,Spot Price(Rs.)_silver,Up/Down_silver,Time_silver,date_silver
0,0,2006-01-02,SILVER,1 KGS,AHMEDABAD,13300,=,12:37:00,2006-01-02
1,1,2006-01-09,SILVER,1 KGS,AHMEDABAD,13500,=,16:50:00,2006-01-09
2,2,2006-01-16,SILVER,1 KGS,AHMEDABAD,13600,=,14:44:00,2006-01-16
3,3,2006-01-23,SILVER,1 KGS,AHMEDABAD,13275,+,12:51:00,2006-01-23
4,4,2006-01-30,SILVER,1 KGS,AHMEDABAD,14170,-,16:56:00,2006-01-30
5,5,2006-02-06,SILVER,1 KGS,AHMEDABAD,13500,-,13:09:00,2006-02-06
6,6,2006-02-13,SILVER,1 KGS,AHMEDABAD,13500,-,12:44:00,2006-02-13
7,7,2006-02-20,SILVER,1 KGS,AHMEDABAD,13700,+,12:36:00,2006-02-20
8,8,2006-02-27,SILVER,1 KGS,AHMEDABAD,13800,=,17:38:00,2006-02-27
9,9,2006-03-06,SILVER,1 KGS,AHMEDABAD,14625,-,16:47:00,2006-03-06


In [292]:
# Copper Price
copper_price_df = combine_data('../Data/Copper')
copper_price_df['date']=copper_price_df['Date'].apply(lambda x: str(x)[0:10])
copper_price_df.drop(columns=['Date'], inplace=True)
copper_price_df = copper_price_df.add_suffix('_copper')
copper_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4088 entries, 0 to 255
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Commodity_copper        4088 non-null   object 
 1   Unit_copper             4088 non-null   object 
 2   Location_copper         4088 non-null   object 
 3   Spot Price(Rs.)_copper  4088 non-null   float64
 4   Up/Down_copper          4088 non-null   object 
 5   Time_copper             4088 non-null   object 
 6   date_copper             4088 non-null   object 
dtypes: float64(1), object(6)
memory usage: 255.5+ KB


In [293]:
copper_price_df = fill_gapes(copper_price_df, date_column='date_copper')
copper_price_df.to_csv('copper_data_expanded.csv')
copper_price_df.sort_values(by='date_copper', inplace=True)
copper_price_df.head(10)

Date Column:  date_copper
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


,Commodity_copper,Unit_copper,Location_copper,Spot Price(Rs.)_copper,Up/Down_copper,Time_copper,date_copper
298,COPPER,1 KGS,MUMBAI,202.9,=,10:50:00,2006-01-02
297,COPPER,1 KGS,MUMBAI,202.9,=,10:50:00,2006-01-03
296,COPPER,1 KGS,MUMBAI,203.0,+,11:36:00,2006-01-04
295,COPPER,1 KGS,MUMBAI,207.4,+,12:09:00,2006-01-05
294,COPPER,1 KGS,MUMBAI,202.9,-,11:04:00,2006-01-06
293,COPPER,1 KGS,MUMBAI,205.4,+,10:46:00,2006-01-07
293,COPPER,1 KGS,MUMBAI,205.4,+,10:46:00,2006-01-08
292,COPPER,1 KGS,MUMBAI,205.4,=,10:53:00,2006-01-09
291,COPPER,1 KGS,MUMBAI,204.7,-,11:10:00,2006-01-10
290,COPPER,1 KGS,MUMBAI,205.8,+,11:10:00,2006-01-11


In [294]:
filtered_copper_price = filter_weekly_data(copper_price_df, 'copper_price', 'date_copper')
filtered_copper_price.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              783 non-null    int64  
 1   monday_date             783 non-null    object 
 2   Commodity_copper        783 non-null    object 
 3   Unit_copper             783 non-null    object 
 4   Location_copper         783 non-null    object 
 5   Spot Price(Rs.)_copper  783 non-null    float64
 6   Up/Down_copper          783 non-null    object 
 7   Time_copper             783 non-null    object 
 8   date_copper             783 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Commodity_copper,Unit_copper,Location_copper,Spot Price(Rs.)_copper,Up/Down_copper,Time_copper,date_copper
0,0,2006-01-02,COPPER,1 KGS,MUMBAI,202.90,=,10:50:00,2006-01-02
1,1,2006-01-09,COPPER,1 KGS,MUMBAI,205.40,=,10:53:00,2006-01-09
2,2,2006-01-16,COPPER,1 KGS,MUMBAI,206.10,=,10:36:00,2006-01-16
3,3,2006-01-23,COPPER,1 KGS,MUMBAI,204.40,=,10:28:00,2006-01-23
4,4,2006-01-30,COPPER,1 KGS,MUMBAI,217.35,=,19:46:00,2006-01-30
5,5,2006-02-06,COPPER,1 KGS,MUMBAI,225.05,=,10:51:00,2006-02-06
6,6,2006-02-13,COPPER,1 KGS,MUMBAI,215.85,=,18:28:00,2006-02-13
7,7,2006-02-20,COPPER,1 KGS,MUMBAI,215.75,-,10:07:00,2006-02-20
8,8,2006-02-27,COPPER,1 KGS,MUMBAI,216.35,=,14:19:00,2006-02-27
9,9,2006-03-06,COPPER,1 KGS,MUMBAI,221.40,=,10:53:00,2006-03-06


In [295]:
# CrudeOil Price
oil_price_df = combine_data('../Data/CrudeOil')
oil_price_df['date']=oil_price_df['Date'].apply(lambda x: str(x)[0:10])
oil_price_df.drop(columns=['Date'], inplace=True)
oil_price_df = oil_price_df.add_suffix('_crudeoil')
oil_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4240 entries, 0 to 1275
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Commodity_crudeoil        4240 non-null   object
 1   Unit_crudeoil             4240 non-null   object
 2   Location_crudeoil         4240 non-null   object
 3   Spot Price(Rs.)_crudeoil  4240 non-null   int64 
 4   Up/Down_crudeoil          4240 non-null   object
 5   Time_crudeoil             4240 non-null   object
 6   date_crudeoil             4240 non-null   object
dtypes: int64(1), object(6)
memory usage: 265.0+ KB


In [296]:
oil_price_df = fill_gapes(oil_price_df, date_column='date_crudeoil')
oil_price_df.to_csv('crudeoil_data_expanded.csv')
oil_price_df.sort_values(by='date_crudeoil', inplace=True)

Date Column:  date_crudeoil
Start Year:  2006
End Year:  2020


<ipython-input-247-2941a36070e4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_row[date_column] = date


In [297]:
filtered_oil_price = filter_weekly_data(oil_price_df, 'crudeoil_price', 'date_crudeoil')
filtered_oil_price.head(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                783 non-null    int64 
 1   monday_date               783 non-null    object
 2   Commodity_crudeoil        783 non-null    object
 3   Unit_crudeoil             783 non-null    object
 4   Location_crudeoil         783 non-null    object
 5   Spot Price(Rs.)_crudeoil  783 non-null    int64 
 6   Up/Down_crudeoil          783 non-null    object
 7   Time_crudeoil             783 non-null    object
 8   date_crudeoil             783 non-null    object
dtypes: int64(2), object(7)
memory usage: 61.2+ KB


,Unnamed: 0,monday_date,Commodity_crudeoil,Unit_crudeoil,Location_crudeoil,Spot Price(Rs.)_crudeoil,Up/Down_crudeoil,Time_crudeoil,date_crudeoil
0,0,2006-01-02,CRUDEOIL,1 BBL,MUMBAI,2755,=,11:16:00,2006-01-02
1,1,2006-01-09,CRUDEOIL,1 BBL,MUMBAI,2816,-,22:09:00,2006-01-09
2,2,2006-01-16,CRUDEOIL,1 BBL,MUMBAI,2830,=,10:48:00,2006-01-16
3,3,2006-01-23,CRUDEOIL,1 BBL,MUMBAI,3032,=,10:53:00,2006-01-23
4,4,2006-01-30,CRUDEOIL,1 BBL,MUMBAI,2982,-,21:21:00,2006-01-30
5,5,2006-02-06,CRUDEOIL,1 BBL,MUMBAI,2891,=,10:35:00,2006-02-06
6,6,2006-02-13,CRUDEOIL,1 BBL,MUMBAI,2731,-,21:28:00,2006-02-13
7,7,2006-02-20,CRUDEOIL,1 BBL,MUMBAI,2662,=,14:31:00,2006-02-20
8,8,2006-02-27,CRUDEOIL,1 BBL,MUMBAI,2754,-,20:56:00,2006-02-27
9,9,2006-03-06,CRUDEOIL,1 BBL,MUMBAI,2805,-,21:02:00,2006-03-06


In [298]:
mon_dates = find_all_mon_dates()
mon_dates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   monday_date  783 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


,monday_date
0,2006-01-02
1,2006-01-09
2,2006-01-16
3,2006-01-23
4,2006-01-30
...,...
778,2020-11-30
779,2020-12-07
780,2020-12-14
781,2020-12-21


In [299]:
## Merging All Dataframes into one

In [300]:
gold_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5478 entries, 290 to 0
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Commodity_gold        5478 non-null   object
 1   Unit_gold             5478 non-null   object
 2   Location_gold         5478 non-null   object
 3   Spot Price(Rs.)_gold  5478 non-null   int64 
 4   Up/Down_gold          5478 non-null   object
 5   Time_gold             5478 non-null   object
 6   date_gold             5478 non-null   object
dtypes: int64(1), object(6)
memory usage: 342.4+ KB


In [301]:
silver_price_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5478 entries, 289 to 0
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Commodity_silver        5478 non-null   object
 1   Unit_silver             5478 non-null   object
 2   Location_silver         5478 non-null   object
 3   Spot Price(Rs.)_silver  5478 non-null   int64 
 4   Up/Down_silver          5478 non-null   object
 5   Time_silver             5478 non-null   object
 6   date_silver             5478 non-null   object
dtypes: int64(1), object(6)
memory usage: 342.4+ KB


In [302]:
from functools import reduce

In [303]:
data_frames=[filtered_inflation_data, filtered_gdp_data, filtered_gni_data, filtered_interest_rate, filtered_uncertainty, filtered_fx_rate, filtered_nifty50, filtered_snp500, filtered_wpm, filtered_ego, filtered_gold_price, filtered_silver_price, filtered_copper_price, filtered_oil_price, filtered_palladium_data, filtered_rhodium_data, filtered_10YrBondRate]
#data_frames=[filtered_inflation_data, filtered_interest_rate,filtered_uncertainty, filtered_fx_rate, filtered_nifty50, filtered_snp500, filtered_wpm, filtered_ego, filtered_gold_price]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['monday_date'],
                                            how='inner'), data_frames)

In [304]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 0 to 782
Columns: 104 entries, Unnamed: 0_x to date_10YrBondRate
dtypes: float64(26), int64(21), object(57)
memory usage: 642.3+ KB


In [305]:
df_merged.to_csv('final_merged_gold_dataset_new.csv')